# Multiclass SVM 구현

In [200]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

#IRIS 데이터 로드
iris =  sns.load_dataset('iris') 
X= iris.iloc[:,:4] #학습할데이터
y = iris.iloc[:,-1] #타겟
print(y)

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


In [201]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) #데이터 분리

In [202]:
def standardization(train, test): #스케일링
    scaler = StandardScaler()
    train = scaler.fit_transform(train)
    test = scaler.transform(test)
    return train, test

X_train, X_test = standardization(X_train, X_test)

In [203]:
class one_vs_rest: 
    def __init__(self, class_number): 
        self.class_number = class_number #class의 갯수
        self.cls = []                    #class의 갯수 만큼 생성되는 classifier를 저장하는 배열
        self.pred = None                 #각 classifier의 prediction 값이 저장되는 배열
        self.result = []                 #argmax를 통해 정해지는 predict class

    def one_hot_encoding(self, y_train): #y를 one-hot-encoding으로 변환
        
        y_train = pd.get_dummies(y_train) 
        return y_train

    def make_model(self, X_train, y_train):
        for i in range(self.class_number):    
            clf = SVC(kernel='rbf', C=5, gamma=5) #class_number의 갯수에 맞게 svc생성
            clf.fit(X_train, y_train.iloc[:,i])   #model fitting
            self.cls.append(clf)                  #각 class에 대한 svc를 cls배열에 저장
    
    def decision_function_predict(self, X_test, labels): 
        cnt =  0 

        for cls in self.cls:                      # cls 배열에서 cls를 받아옴
            if self.pred is None:                 
                self.pred = cls.decision_function(X_test) # 만약 첫 cls라서 배열값이 없으면 해당 cls의 pred를 추가함. 
            
            else: #이미 self.pred가 차있다면 stack을 저장
                self.pred = np.vstack((self.pred, cls.decision_function(X_test)))

        self.pred = self.pred.T #전치로 뒤집으면 배열의 첫번째 줄은 각 cls에 대한 pred값이 나타나게됨. 
        for value in self.pred:
            
            for temp in value: #각 줄마다의 음수값의 개수를 알려고함 
                if temp<0: 
                    cnt = cnt + 1 

            if cnt == len(value):  #만약 줄의 모든 요소들이 음수라면
                self.result.append(labels[value.argmin()]) # 가장 작은 음수(절대값상으로 가장큰)를 추가 
            
            else : 
                self.result.append(labels[value.argmax()]) # 그 외에는 가장 큰 수를 추가(양수)
            
        return self.result

    def test(self, y_test):
        print(accuracy_score(y_test, self.result))


In [204]:
one_vs_rest = one_vs_rest(3)
y_train =one_vs_rest.one_hot_encoding(y_train)
one_vs_rest.make_model(X_train, y_train)

In [205]:
labels = ["setosa", "versicolor", "virginica"]
pred =one_vs_rest.decision_function_predict(X_test, labels)

In [206]:
one_vs_rest.test(y_test)

0.9333333333333333
